<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Preamble" data-toc-modified-id="Preamble-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Preamble</a></span></li><li><span><a href="#Different-Regions" data-toc-modified-id="Different-Regions-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Different Regions</a></span></li><li><span><a href="#Functions" data-toc-modified-id="Functions-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Functions</a></span></li><li><span><a href="#Indivual-Phases" data-toc-modified-id="Indivual-Phases-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Indivual Phases</a></span></li><li><span><a href="#Bootstrap" data-toc-modified-id="Bootstrap-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Bootstrap</a></span><ul class="toc-item"><li><span><a href="#Create-Count-Array" data-toc-modified-id="Create-Count-Array-5.1"><span class="toc-item-num">5.1&nbsp;&nbsp;</span>Create Count Array</a></span></li><li><span><a href="#Functions" data-toc-modified-id="Functions-5.2"><span class="toc-item-num">5.2&nbsp;&nbsp;</span>Functions</a></span></li><li><span><a href="#Running-and-Saving" data-toc-modified-id="Running-and-Saving-5.3"><span class="toc-item-num">5.3&nbsp;&nbsp;</span>Running and Saving</a></span></li></ul></li></ul></div>

# Preamble

In [4]:
##############################

import xarray as xr
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import dask.array
import cartopy.crs as ccrs
import pickle
import matplotlib.colors as colors
import datetime as dt
rb = plt.cm.RdBu
bm = plt.cm.Blues

import warnings
warnings.filterwarnings('ignore')

/g/data3/hh5/public/apps/miniconda3/envs/analysis3-19.01/lib/python3.6/site-packages/dask/config.py:168: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  data = yaml.load(f.read()) or {}
/g/data3/hh5/public/apps/miniconda3/envs/analysis3-19.01/lib/python3.6/site-packages/distributed/config.py:20: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  defaults = yaml.load(f)


In [5]:
path = 'RMM.pickle'
pickle_in = open(path, 'rb')
RMM = pickle.load(pickle_in)

In [6]:
# %load /home/563/ab2313/MJO/get_awap.py
import sys

def get_platform():
    platforms = {
        'linux1' : 'Linux',
        'linux2' : 'Linux',
        'darwin' : 'OS X',
        'win32' : 'Windows'
    }

    if sys.platform not in platforms:
        return sys.platform

    return platforms[sys.platform]


platform = get_platform()


if platform == 'OS X':
    path =  '/Users/alexborowiak/Desktop/large_files/'


   
    
else:
#     path = '/home/student.unimelb.edu.au/aborowiak/Desktop/Code/Scripts/big_files/'
    path = '/home/563/ab2313/big_files/'


# precip = xr.open_dataset(path + 'AWAP_w.nc')


In [7]:

precip = xr.open_dataset(path + 'AWAP_W.nc', chunks={'time':-1, 'lat': 50, 'lon': 50}).precip

In [8]:
RMM = RMM.reset_index()
RMM['Date'] = RMM['Date'] + pd.to_timedelta('9h')
RMM = RMM.set_index('Date')

# Different Regions

The rainfall is divided into different phases

In [9]:
regions = np.array([slice(110, 120),slice(120.25, 140),slice(140.25, 156.25)])
mjo_enhanced_list = np.array([[4,5],[4,5,6],[4,5,6,7]])

In [10]:
# This is creating 3 seperate data frames comprised of the rainfall just in the individual phases


########## Enhanced

region_data = []

for reg_num in [0,1,2]:
    region = regions[reg_num]
    precip_region = precip.sel(lon = region)

    mjo_enhanced = mjo_enhanced_list[reg_num]
    enhanced_dates = np.array(RMM[np.logical_and(RMM['Phase'].isin(mjo_enhanced), RMM['Amplitude'] >= 1)].index)

    precip_enhancecd = precip_region.where(precip_region.time.isin(enhanced_dates))
    
    region_data.append(precip_enhancecd)
    
enhanced_precip = (region_data[0].combine_first(region_data[1])).combine_first(region_data[2])



########## Supppressed
region_data = []

for reg_num in [0,1,2]:
    region = regions[reg_num]
    precip_region = precip.sel(lon = region)

    mjo_suppressed = mjo_enhanced_list[reg_num]
    suppressed_dates = np.array(RMM[np.logical_and(~RMM['Phase'].isin(mjo_suppressed), RMM['Amplitude'] >= 1)].index)

    precip_enhancecd = precip_region.where(precip_region.time.isin(suppressed_dates))
    
    region_data.append(precip_enhancecd)
    
suppressed_precip = (region_data[0].combine_first(region_data[1])).combine_first(region_data[2])

########## Inactive
region_data = []

for reg_num in [0,1,2]:
    region = regions[reg_num]
    precip_region = precip.sel(lon = region)

    mjo_inactive = mjo_enhanced_list[reg_num]
    inactive_dates = np.array(RMM[RMM['Amplitude'] <1].index)

    precip_enhancecd = precip_region.where(precip_region.time.isin(inactive_dates))
    
    region_data.append(precip_enhancecd)
    
inactive_precip = (region_data[0].combine_first(region_data[1])).combine_first(region_data[2])

In [11]:
# The climatology for each month. The values above this are considered to be extreme. This will be what I will
# be counting. This is the 90th percentile of all rainfall

climatology_90 = precip.groupby('time.month').reduce(np.nanpercentile, q = 90, dim = 'time')
climatology_95 = precip.groupby('time.month').reduce(np.nanpercentile, q = 95, dim = 'time')

The values greater than the 90th percentile from each phase of the MJO is returned

In [12]:
# This returns just the extremes. 

def return_extremes(precip, threshold):
    storage = []
    months = [10,11,12,1,2,3]
    for i,month in enumerate(months):
        precip_month = precip.where(precip.time.dt.month == month, drop = True)
        threshold_month = threshold.sel(month = month)
        
        precip_month_ex = precip_month.where(precip_month >= threshold_month)

        storage.append(precip_month_ex)
        
        if i == 0:
            extreme_xr = precip_month_ex
        else:
            extreme_xr = extreme_xr.combine_first(precip_month_ex)  
    
    return  extreme_xr
    

In [13]:
enhanced_90 = return_extremes(enhanced_precip , climatology_90)
suppressed_90 = return_extremes(suppressed_precip , climatology_90)
inactive_90 = return_extremes(inactive_precip , climatology_90)

precip_90 = return_extremes(precip , climatology_90)

In [14]:
enhanced_95 = return_extremes(enhanced_precip , climatology_95)
suppressed_95 = return_extremes(suppressed_precip , climatology_95)
inactive_95 = return_extremes(inactive_precip , climatology_95)

precip_95 = return_extremes(precip , climatology_95)

# Functions

In [15]:
def into_xr(data, orgininal, name = 'precip'):
    return xr.DataArray(
    data,
    dims=['lat','lon'],
    coords={'lat':orgininal.lat, 'lon': orgininal.lon},
    name=name
    )

In [16]:
def calculate_sumfrac(data_sub,data_all):
    
    # suming all of the extreme events
    sum_sub = data_sub.sum(dim = 'time')
    
    # suming all of the rain events
    sum_all = data_all.sum(dim = 'time')
    
    # Finding the ratio of the events
    frac = sum_sub/sum_all
    
    
    xr = into_xr(frac, data_all, 'precip')
    
    return xr

In [17]:
def sum_all_months_and_regions(phase_precip_ex, phase_precip, precip_ex, precip):
    
    months = [10,11,12,1,2,3]
    temp_dict = {}
    
    for month in months:
        ###Just the data that is in this month
        # For the MJO Phase
        phase_prec_month = phase_precip_ex.where(phase_precip_ex.time.dt.month == month, drop = True)
        all_prec_month = phase_precip.where(phase_precip.time.dt.month == month, drop = True)
        # For all rainfall 
        prec_month = precip_ex.where(precip_ex.time.dt.month == month, drop = True)
        prec_month = precip.where(precip.time.dt.month == month, drop = True)
        
        
        ##### Running the function above
        # The fracion of extremes compared to normal rainfall for MJO phase
        frac_phase = calculate_sumfrac(phase_prec_month ,all_prec_month)  
        # The fracion of extremes compared to normal rainfall
        frac_all = calculate_sumfrac(prec_month , prec_month)
        
        # Dividing these two will give the comparison (e.g twice as much)
        frac_month = frac_phase/ frac_all
        
        # Storing the results
        temp_dict[str(month)] = frac_month
    
    frac_int = xr.concat([temp_dict['10'],temp_dict['11'],temp_dict['12'],temp_dict['1'],
                            temp_dict['2'],temp_dict['3']], pd.Index(months, name = 'month'))
    
    return frac_int
    
    
    

# Indivual Phases

In [18]:

enhanced_sum90 = sum_all_months_and_regions(enhanced_90 , enhanced_precip, precip_90, precip)

suppressed_sum90 = sum_all_months_and_regions(suppressed_90, suppressed_precip, precip_90, precip)

inactive_sum90 = sum_all_months_and_regions(inactive_90, inactive_precip, precip_90, precip)

In [19]:
prop_90 = xr.concat([enhanced_sum90, suppressed_sum90, inactive_sum90],
                    pd.Index(['enhanced','suppressed','inactive'], name = 'mjo'))

In [20]:

enhanced_sum95 = sum_all_months_and_regions(enhanced_95 , enhanced_precip, precip_95, precip)

suppressed_sum95 = sum_all_months_and_regions(suppressed_95, suppressed_precip, precip_95, precip)

inactive_sum95 = sum_all_months_and_regions(inactive_95, inactive_precip, precip_95, precip)

In [21]:
prop_95 = xr.concat([enhanced_sum95, suppressed_sum95, inactive_sum95],
                    pd.Index(['enhanced','suppressed','inactive'], name = 'mjo'))

In [22]:
save = 0
if save:
    prop_95.to_netcdf('prop_95.nc')

    prop_90.to_netcdf('prop_90.nc')

# Bootstrap

Steps:

* 1: Creating the count array. This counts all of the rainfall events that occur. So this will count all in the normal and the extreme arrays. This is useful as now I only have to do a sum across all of the years (months).
* 2: Th e

In [23]:
enhanced_precip

<xarray.DataArray 'precip' (time: 7822, lat: 53, lon: 178)>
dask.array<shape=(7822, 53, 178), dtype=float32, chunksize=(7822, 50, 50)>
Coordinates:
  * lon      (lon) float64 112.0 112.2 112.5 112.8 ... 155.5 155.8 156.0 156.2
  * time     (time) datetime64[ns] 1974-10-01T09:00:00 ... 2017-12-31T09:00:00
  * lat      (lat) float64 -23.0 -22.75 -22.5 -22.25 ... -10.5 -10.25 -10.0
Attributes:
    long_name:  Daily Precipitation
    units:      mm

In [24]:
enhanced_90

<xarray.DataArray 'precip' (time: 7822, lat: 53, lon: 178)>
dask.array<shape=(7822, 53, 178), dtype=float32, chunksize=(7822, 50, 50)>
Coordinates:
  * time     (time) datetime64[ns] 1974-10-01T09:00:00 ... 2017-12-31T09:00:00
  * lon      (lon) float64 112.0 112.2 112.5 112.8 ... 155.5 155.8 156.0 156.2
  * lat      (lat) float64 -23.0 -22.75 -22.5 -22.25 ... -10.5 -10.25 -10.0
Attributes:
    long_name:  Daily Precipitation
    units:      mm

## Create Count Array

The count array is created to speed up the bootstrapping.
As the end results of count is the count from every mouth, 
if the count of each month is already created, then the final 
results can just be the sum of all of these.

In [25]:
def each_year_sum(data):
    
    storage = []
    #Looping through all of the months
    months = [10,11,12,1,2,3]
    for month in months:
        data_month = data.where(data.time.dt.month == month, drop = True)
        sum_year = data_month.groupby('time.year').sum(dim = 'time')
        storage.append(sum_year)
        
    sum_all = xr.concat(storage, pd.Index(months, name = 'month'))
    
    return sum_all
        
        


In [26]:
precip_90

<xarray.DataArray 'precip' (time: 7822, lat: 53, lon: 178)>
dask.array<shape=(7822, 53, 178), dtype=float32, chunksize=(7822, 50, 50)>
Coordinates:
  * time     (time) datetime64[ns] 1974-10-01T09:00:00 ... 2017-12-31T09:00:00
  * lon      (lon) float64 112.0 112.2 112.5 112.8 ... 155.5 155.8 156.0 156.2
  * lat      (lat) float64 -23.0 -22.75 -22.5 -22.25 ... -10.5 -10.25 -10.0
Attributes:
    long_name:  Daily Precipitation
    units:      mm

In [27]:
precip_year_sum_90 = each_year_sum(precip_90)
precip_year_sum_95 = each_year_sum(precip_95)
precip_year_sum = each_year_sum(precip)


In [28]:
en_90_year_sum = each_year_sum(enhanced_90)
sup_90_year_sum = each_year_sum(suppressed_90)
inact_90_year_sum = each_year_sum(inactive_90)

mjo_year_sum_90 = xr.concat([en_90_year_sum, sup_90_year_sum,inact_90_year_sum],
                       pd.Index(['enhanced', 'suppressed','inactive'], name = 'mjo'))

In [29]:
en_95_year_sum = each_year_sum(enhanced_95)
sup_95_year_sum = each_year_sum(suppressed_95)
inact_95_year_sum = each_year_sum(inactive_95)

mjo_year_sum_95 = xr.concat([en_95_year_sum, sup_95_year_sum,inact_95_year_sum],
                       pd.Index(['enhanced', 'suppressed','inactive'], name = 'mjo'))

In [30]:
en_year_sum = each_year_sum(enhanced_precip)
sup_year_sum = each_year_sum(suppressed_precip)
inact_year_sum = each_year_sum(inactive_precip)

mjo_year_sum = xr.concat([en_year_sum, sup_year_sum,inact_year_sum],
                       pd.Index(['enhanced', 'suppressed','inactive'], name = 'mjo'))

## Functions

In [31]:
def select_additional(data_sub,data,uniqueValues, occurCount, number):

    
    if any(occurCount >= 2 ):
        # Find the years that are getting sampled multiple times
            # occurCount is the frequency of each year
            # uniqueValues is all the years that have been sampled
        multi_sampled_years = uniqueValues[np.where(occurCount >= 2 )]
        
        #  Finding the data for the additional years
        additional = data.where(data.year.isin(multi_sampled_years), drop = True)
        # Adding n * 100 years to the data so that they don't overlap
        additional['year'] = additional.year +  number * 100
        
        # Combing all the data together
        data_sub = data_sub.combine_first(additional)
        
        #### Running this whole function again (recursivly) to determine which ones have been selected more than
           # twice
        # Reducing the occur count by 1. This will enable the samples that have been sampeld more than twice 
        # to be determing
        occurCount = occurCount - 1
        number += 1
        
        data_sub = select_additional(data,data_sub, uniqueValues, occurCount, number) 
        
        return data_sub
                                
    else:
        return data_sub

In [32]:
def calculate_sum_of_sum_frac(data_sub,data_all):
    
    # suming all of the extreme events
    sum_sub = data_sub.sum(dim = 'year')
    
    # suming all of the rain events
    sum_all = data_all.sum(dim = 'year')
    
    # Finding the ratio of the events
    frac = sum_sub/sum_all
    
    return frac

In [33]:
def sum_all_months_and_regions(phase_precip_ex, phase_precip, precip_ex, precip):
    
    months = [10,11,12,1,2,3]
    temp_dict = {}
    
    for month in months:
        ###Just the data that is in this month
        # For the MJO Phase
        phase_prec_month = phase_precip_ex.where(phase_precip_ex.time.dt.month == month, drop = True)
        all_prec_month = phase_precip.where(phase_precip.time.dt.month == month, drop = True)
        # For all rainfall 
        prec_month = precip_ex.where(precip_ex.time.dt.month == month, drop = True)
        prec_month = precip.where(precip.time.dt.month == month, drop = True)
        
        
        ##### Running the function above
        # The fracion of extremes compared to normal rainfall for MJO phase
        frac_phase = calculate_sumfrac(phase_prec_month ,all_prec_month)  
        # The fracion of extremes compared to normal rainfall
        frac_all = calculate_sumfrac(prec_month , prec_month)
        
        # Dividing these two will give the comparison (e.g twice as much)
        frac_month = frac_phase/ frac_all
        
        # Storing the results
        temp_dict[str(month)] = frac_month
    
    frac_int = xr.concat([temp_dict['10'],temp_dict['11'],temp_dict['12'],temp_dict['1'],
                            temp_dict['2'],temp_dict['3']], pd.Index(months, name = 'month'))
    
    return frac_int
    

In [34]:
# This function takes in the sum for a single phase of the MJO

def bootstrap_each_month_sum(phase_sum_ex, phase_sum, sum_ex, sum_p, loops):
    
    month_storage = []
    
    # Loop through all of the months. Extract the data for just one month.
    months = [10,11,12,1,2,3]
    for month in months:
        
        loop_storage = []
        
        # The MJO Phase
        phase_sum_month_ex = phase_sum_ex.sel(month = month)
        phase_sum_month = phase_sum.sel(month = month)
        
        # All Rainfall
        sum_month_ex = sum_ex.sel(month = month)
        sum_month = sum_p.sel(month = month)
        
        # Looping through all of the different runs
        for i in range(loops):
            
            ########### Selecting Random Data
            # Selecting random years; half the ranges of the data
            rand_years = np.random.randint(1974,2017,22)
    
            # Subsetting botht the data to the randomyears
            phase_sum_ex_rand = phase_sum_month_ex.where(phase_sum_month_ex.year.isin(rand_years)) # Extremes
            phase_sum_rand = phase_sum_month.where(phase_sum_month.year.isin(rand_years)) # Normal
        
            sum_ex_rand = sum_month_ex.where(sum_month_ex.year.isin(rand_years)) # Extremes
            sum_rand = sum_month.where(sum_month.year.isin(rand_years)) # Normal
            
            # If there are years that have been repeated they need to be added
            uniqueValues, occursum = np.unique(rand_years, return_counts=True)
            
            # (data_sub, data)
            phase_sum_ex_rand = select_additional(phase_sum_ex_rand,phase_sum_month_ex, uniqueValues, occursum, number = 1)
            phase_sum_rand = select_additional(phase_sum_rand, phase_sum_month,uniqueValues, occursum, number = 1)
            
            sum_ex_rand = select_additional(sum_ex_rand,sum_month_ex, uniqueValues, occursum, number = 1)
            sum_rand = select_additional(sum_rand, sum_month,uniqueValues, occursum, number = 1)
            
            
            ########### Actual Calculation
            # (data_sub, data_all)
            # For the phase: the proportion of extremes
            frac_phase = calculate_sum_of_sum_frac(phase_sum_ex_rand, phase_sum_rand)
            
            # For all rainfall: the proportion of extremes
            frac_all = calculate_sum_of_sum_frac(sum_ex_rand, sum_rand)
            
            boot_run = frac_phase/frac_all
            
            loop_storage.append(boot_run)
            
            ## END OF LOOP
        
        month_data = xr.concat(loop_storage, 'run_num')
        month_95 = month_data.reduce(np.nanpercentile, q = 95, dim = 'run_num')
        month_5 = month_data.reduce(np.nanpercentile, q = 5, dim = 'run_num')
        
        month_boot = xr.concat([month_95, month_5], pd.Index([95,5], name = 'percentile'))
        month_storage.append(month_boot)
        
    total_boot = xr.concat(month_storage, pd.Index(months, name = 'month'))
        
    return total_boot
    
    
    

## Running and Saving

In [35]:
loops = 1000



name = 'enhanced'
enhanced_boot = bootstrap_each_month_sum(mjo_year_sum_90.sel(mjo = name),mjo_year_sum.sel(mjo = name),
                                           precip_year_sum_90 ,precip_year_sum,loops)

name = 'suppressed'
suppressed_boot = bootstrap_each_month_sum(mjo_year_sum_90.sel(mjo = name),mjo_year_sum.sel(mjo = name),
                                           precip_year_sum_90 ,precip_year_sum,loops)

name = 'inactive'
inactive_boot = bootstrap_each_month_sum(mjo_year_sum_90.sel(mjo = name),mjo_year_sum.sel(mjo = name),
                                           precip_year_sum_90 ,precip_year_sum,loops)



In [36]:
prop_boot_90 = xr.concat([enhanced_boot, suppressed_boot, inactive_boot], 
                        pd.Index(['enhanced', 'suppressed','inactive'], name = 'mjo'))

prop_boot_90.to_netcdf('prop_boot_90.nc')

In [37]:


name = 'enhanced'
enhanced_boot = bootstrap_each_month_sum(mjo_year_sum_95.sel(mjo = name),mjo_year_sum.sel(mjo = name),
                                           precip_year_sum_95 ,precip_year_sum,loops)

name = 'suppressed'
suppressed_boot = bootstrap_each_month_sum(mjo_year_sum_95.sel(mjo = name),mjo_year_sum.sel(mjo = name),
                                           precip_year_sum_95 ,precip_year_sum,loops)

name = 'inactive'
inactive_boot = bootstrap_each_month_sum(mjo_year_sum_95.sel(mjo = name),mjo_year_sum.sel(mjo = name),
                                           precip_year_sum_95 ,precip_year_sum,loops)


In [38]:
prop_boot_95 = xr.concat([enhanced_boot, suppressed_boot, inactive_boot], 
                        pd.Index(['enhanced', 'suppressed','inactive'], name = 'mjo'))

prop_boot_95.to_netcdf('prop_boot_95.nc')